In [ ]:
!pip install ibm-cloud-sdk-core --no-cache | tail -n 1
!pip install --upgrade ibm-watson-machine-learning --user | tail -n 1

In [ ]:
import os
token = os.environ['USER_ACCESS_TOKEN']
url = os.getenv('RUNTIME_ENV_APSX_URL')

WML_CREDENTIALS = {
   "token" : token,
   "instance_id" : "openshift",
   "url": url,
   "version": "3.5"
}

In [ ]:
import json
from ibm_watson_machine_learning import APIClient

wml_client = APIClient(WML_CREDENTIALS)
wml_client.version

In [ ]:
wml_client.spaces.list(limit=10)

In [ ]:
spaces = wml_client.spaces.get_details()['resources']
space_id = None
for space in spaces:
    if space['entity']['name'] == 'Attrition Production Space':
        space_id = space["metadata"]["id"]
wml_client.set.default_space(space_id)

In [ ]:
for model in wml_client.deployments.get_details()['resources']:
    if model['metadata']['name'] == 'predictive attrition - 2018':
        deployment_id = model['metadata']['id']
deployment_id

In [ ]:
import pandas as pd

df_data_1 = pd.read_csv('/project_data/data_asset/autoai_records_2020.csv')
df_data_1.head()

In [ ]:
feed_data = df_data_1.drop(['ATTRITION'], axis=1).sample(n=100)

In [ ]:
feed_data.head()

In [ ]:
fields = feed_data.columns.tolist()
values = feed_data.values.tolist()

payload_scoring = {"input_data": [{"fields": fields, "values": values}]}  

In [ ]:
predictions = wml_client.deployments.score(deployment_id, payload_scoring)
predictions